In [22]:
####################################
#ENVIRONMENT SETUP

In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [2]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [3]:
#IMPORT CLASSES (from current directory)
sys.path.append(codeDirectory)
from CLASSES_CalculateMoreVariables import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [4]:
####################################
#LOADING CLASSES

In [5]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str, dataName="Eulerian_Binary_Array", dtype='bool')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 outputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projec

In [16]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:7


In [17]:
####################################
#FUNCTIONS

In [18]:
#Calculation Functions
def GetCloudyUpdraftThresholds():
    w_thresh1=0.1
    w_thresh2=0.5
    qcqi_thresh=1e-6
    return w_thresh1,w_thresh2,qcqi_thresh

def GetInputVariables(inputDataDirectory, timeString):
    w_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="winterp")
    qc_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qc")
    qi_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qi")
    qcqi_data=qc_data+qi_data

    InputDictionary = {'w_data': w_data,
                       'qcqi_data': qcqi_data}
    return InputDictionary
    
def VariableCalculation(inputDictionary):
    [w_data, qcqi_data] = (inputDictionary[k] for k in ['w_data', 'qcqi_data'])
    [w_thresh1,w_thresh2,qcqi_thresh] = GetCloudyUpdraftThresholds()

    #calculating A_g binary array
    cond1 = w_data >= w_thresh1
    cond2 = qcqi_data < qcqi_thresh
    A_g = np.where(cond1 & cond2, True, False)

    #calculating A_c binary array
    cond1 = w_data >= w_thresh2
    cond2 = qcqi_data >= qcqi_thresh
    A_c = np.where(cond1 & cond2, True, False)

    outputDictionary = {
        'A_c': A_c,
        'A_g': A_g
    }
    return outputDictionary

In [19]:
####################################
#RUNNING

In [21]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    inputDictionary = GetInputVariables(DataManager.inputDataDirectory, timeString)

    #calculating variables
    outputDictionary = VariableCalculation(inputDictionary)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, ModelData.res, ModelData.t_res, timeString, outputDictionary)

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Array_1km_5min_34nz_0-00-00.h5
Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Array_1km_5min_34nz_0-05-00.h5
Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Array_1km_5min_34nz_0-10-00.h5
Current time 3
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Arr

In [ ]:
######################################################

In [8]:
# #READING BACK IN
# def GetVariableData(t, dataName="Eulerian_Binary_Array"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataName)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)

Keys in file: ['A_c', 'A_g']
A_c: shape = (34, 200, 512), dtype = bool
A_g: shape = (34, 200, 512), dtype = bool
